In [1]:
import pandas as pd
import numpy as np
import os

# DATA INGESTION

LOAD DATA FROM DATA FILES

In [2]:
# Global variables
data = {}
data_names = []
non_file = []
base_folder = "./../pseudo_data"

DECLARE ALL INGESTION FUNCTIONS

In [ ]:
def isFile(path):
    global non_file
    if os.path.isfile(path):
        return True
    else:
        non_file.append(path)
        return False


def getAllFilesInDirectory(directory):
    """
    Get all files in a directory.
    """
    global non_file
    data = [os.path.join(directory + "/", f) for f in os.listdir(directory) if isFile(os.path.join(directory + "/", f))]

    while len(non_file) > 0:
        data.extend(getAllFilesInDirectory(non_file.pop(0)))

    return data


# Get data from files
def getDataFromFile(files):
    """
    Get data from a file.
    """
    # global data_names, data
    data_names_, data_ = [], {}

    def getDataFromFile_(file):
        if file.endswith(".csv"):
            return pd.read_csv(file)
        elif file.endswith(".json"):
            return pd.read_json(file)
        elif file.endswith(".xlsx"):
            return pd.read_excel(file)
        else:
            raise ValueError("Unsupported file format")

    for file in files:
        nom = file[file.rfind("/")+1:-4]
        data_names_.append(nom)
        data_[nom] = getDataFromFile_(file)
        print(f"⬇️ Loading `{nom}`   ---   {file}")
        
    print("\n", "✅ All data loaded successfully")
    return (data_names_, data_)
    
    # return data[data_names[0]].info(), data[data_names[0]].head(), data[data_names[0]].describe(), data[data_names[0]].isnull().sum(), data[data_names[0]].dtypes, data[data_names[0]].columns, data[data_names[0]].shape, data[data_names[0]].nunique(), data[data_names[0]].duplicated().sum(), data[data_names[0]].sample(5), data[data_names[0]].sample(5).to_dict(), data[data_names[0]].sample(5).to_json()
    # return data[data_names[0]].info(), data[data_names[0]].describe(), data[data_names[0]].isnull().sum(), data[data_names[0]].dtypes, data[data_names[0]].columns, data[data_names[0]].shape, data[data_names[0]].nunique(), data[data_names[0]].duplicated().sum(), data[data_names[0]].sample(5), data[data_names[0]].sample(5).to_dict(), data[data_names[0]].sample(5).to_json()


In [4]:
# Get all files in the directory
files = getAllFilesInDirectory(base_folder)
# print(files)
(data_names, data) = getDataFromFile(files)
# print(data_names)

Loading admins   ---   ./../pseudo_data/admins.csv
Loading analytics_cache   ---   ./../pseudo_data/analytics_cache.csv
Loading audit_logs   ---   ./../pseudo_data/audit_logs.csv
Loading companies   ---   ./../pseudo_data/companies.csv
Loading inventory   ---   ./../pseudo_data/inventory.csv
Loading items   ---   ./../pseudo_data/items.csv
Loading permissions   ---   ./../pseudo_data/permissions.csv
Loading sales   ---   ./../pseudo_data/sales.csv
Loading sale_items   ---   ./../pseudo_data/sale_items.csv
Loading stock_entries   ---   ./../pseudo_data/stock_entries.csv
Loading system_roles   ---   ./../pseudo_data/system_roles.csv
Loading system_users   ---   ./../pseudo_data/system_users.csv
Loading transaction_logs   ---   ./../pseudo_data/transaction_logs.csv
Loading users   ---   ./../pseudo_data/users.csv
Loading user_roles   ---   ./../pseudo_data/user_roles.csv
Data loaded successfully


FORMAT DATA (REMOVE id FROM ALL DATASET)

In [5]:
# from datetime import datetime, timedelta

In [ ]:
def removeAllDatasetIds(dataset, names):
    for name in names:
        dataset[name] = dataset[name].iloc[:, 1:]

    print("✅ Data Transformed Successfully!")
    return dataset

In [7]:
# data = removeAllDatasetIds(data, data_names)

LOAD DATA TO DATABASE

In [ ]:
import yaml
import logging
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
logging = logging.basicConfig(filename="../capstone.log", level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
engine = None # To initialise the engine globally

def databaseConnection():
    global engine
    if engine is None:
        try:
            with open('conn.yaml', mode='r') as file:
                config = yaml.safe_load(file)

                db_url = sqlalchemy.URL.create(
                    drivername='postgresql+psycopg2', 
                    username=config['user'], 
                    password=config['password'], 
                    host=config['host'], 
                    port=config['port'], 
                    database=config['database']
                )
                engine = sqlalchemy.create_engine(db_url)
                print("✅ Connected Successfully!", "\n")
        except Exception as e:
            print(f"❌ Failed to connect to database: {e}")
            raise
        
    return engine

In [ ]:
def uploadToDatabase(dataset, names, engine):
    for name in names:
        try:
            dataset[name].to_sql(name=name, con=engine, if_exists='append', index=False)
            print(f"✅ Uploaded `{name}` dataset successfully.")
        except SQLAlchemyError as e:
            print(f"❌ Failed to upload `{name}` dataset: {e}")

In [11]:
databaseConnection()
uploadToDatabase(data, data_names, engine)

Connected Successfully!
✅ Uploaded `admins` successfully.
✅ Uploaded `analytics_cache` successfully.
✅ Uploaded `audit_logs` successfully.
✅ Uploaded `companies` successfully.
✅ Uploaded `inventory` successfully.
✅ Uploaded `items` successfully.
✅ Uploaded `permissions` successfully.
✅ Uploaded `sales` successfully.
✅ Uploaded `sale_items` successfully.
✅ Uploaded `stock_entries` successfully.
✅ Uploaded `system_roles` successfully.
❌ Failed to upload `system_users`
✅ Uploaded `transaction_logs` successfully.
✅ Uploaded `users` successfully.
✅ Uploaded `user_roles` successfully.


# 